<a href="https://colab.research.google.com/github/Hajar-lyoubi/AI54_TD_Tutorials/blob/main/TD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TD 03 - Exercises - Word Embeddings**


  # *0- Basics of neural networks with PyTorch*


In this section, we build a simple neural network to understand the basic workflow in PyTorch:
- loading a dataset,
- defining a model,
- training it, and
- evaluating its accuracy.

The dataset `height_weight_sex_training_set.csv` contains people’s height, weight, and sex.  
The goal is to predict whether a person is **male** or **female** based on height and weight.


### Step 1 — Importing libraries

---



In [1]:

import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd

### Step 2 — Loading and inspecting the dataset  


---



In [2]:
df = pd.read_csv('height_weight_sex_training_set.csv')
df.head()

,Height,Weight,Sex
0,165.65,35.41,Female
1,148.53,74.45,Female
2,167.04,81.22,Male
3,161.54,71.47,Male
4,174.31,78.18,Male


The dataset looks correct: each row corresponds to one individual with their height, weight, and gender.  
Next, we will prepare the data to be used as input tensors for the neural network.


### Step 3 — Preparing the input data  


---

In this step, we prepare the **inputs** (features) for the neural network.  
We extract the columns `Height` and `Weight` from the dataset and convert them into PyTorch tensors of type `float32`.  
Each individual is represented by a vector of size 2 → `[height, weight]`.

We use the function `unsqueeze(1)` to add an extra dimension, making sure the data has the right shape for concatenation.  
Finally, we concatenate both tensors into a single input matrix.


In [3]:
# Preparing the inputs (we want a list of vectors of size 2)
heights = torch.tensor(df['Height'], dtype=torch.float32).unsqueeze(1)
weights = torch.tensor(df['Weight'], dtype=torch.float32).unsqueeze(1)
inputs = torch.cat((heights, weights), dim=1)


### Step 4 — Preparing the output data  


---



Next, we prepare the **outputs** (labels) for training.  
The column `Sex` contains categorical values ('Male' or 'Female'),  
so we replace these strings by numeric labels: 0 for *Female* and 1 for *Male*.

In [4]:

# Preparing the outputs (we want 1-hot encoded values for the two possible classes)
outputs = F.one_hot(torch.tensor(df['Sex'].replace('Female', 0).replace('Male', 1))).float()

/tmp/ipython-input-47438714.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  outputs = F.one_hot(torch.tensor(df['Sex'].replace('Female', 0).replace('Male', 1))).float()


### Step 5 — Defining the neural network model  


---


In this step, we define a very simple neural network using PyTorch.  
The model is **sequential**, meaning each layer feeds its output to the next one.  

- The **input layer** receives 2 values (height and weight).  
- The **hidden layer** has 16 neurons with a linear transformation followed by a ReLU activation (implicitly handled later).  
- The **output layer** has 2 neurons, one for each possible category: *Male* or *Female*.  


In [6]:
# Defining the model (i.e. the neural network)
model = nn.Sequential(
    nn.Linear(2, 16),
    nn.Linear(16, 2)
)

### Step 6 — Training the neural network  


---
Now that the model is defined, we can train it using the prepared data.  
We define:
- **Loss function:** `CrossEntropyLoss()` — compares predicted classes with true labels.  
- **Optimizer:** `Adam` — adjusts weights to minimize the loss function.  

The training loop runs for several epochs (iterations).  
At each epoch:
1. The model predicts the outputs (`logits`),
2. The loss is computed,
3. The gradients are backpropagated,
4. The optimizer updates the model parameters.

We print the loss every 10 epochs to monitor convergence.


In [8]:
# Training the model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 2000
for epoch in range(1, epochs + 1):
  logits = model(inputs)                # The model is applied on all the inputs
  loss = criterion(logits, outputs)     # The error is computed for all the predictions (logits) according to expected outputs

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  # Every 10 step we print the epoch and the loss so we can see the training
  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss}")

Epoch: 10, Loss: 0.8577123284339905
Epoch: 20, Loss: 1.0777045488357544
Epoch: 30, Loss: 0.4793497622013092
Epoch: 40, Loss: 0.5959108471870422
Epoch: 50, Loss: 0.5192997455596924
Epoch: 60, Loss: 0.4882560968399048
Epoch: 70, Loss: 0.47098231315612793
Epoch: 80, Loss: 0.46969014406204224
Epoch: 90, Loss: 0.4675212502479553
Epoch: 100, Loss: 0.46719974279403687
Epoch: 110, Loss: 0.466734915971756
Epoch: 120, Loss: 0.46631932258605957
Epoch: 130, Loss: 0.4659329950809479
Epoch: 140, Loss: 0.46551135182380676
Epoch: 150, Loss: 0.46510863304138184
Epoch: 160, Loss: 0.46470406651496887
Epoch: 170, Loss: 0.4643022119998932
Epoch: 180, Loss: 0.46390438079833984
Epoch: 190, Loss: 0.4635111391544342
Epoch: 200, Loss: 0.46312370896339417
Epoch: 210, Loss: 0.46274250745773315
Epoch: 220, Loss: 0.46236860752105713
Epoch: 230, Loss: 0.4620024859905243
Epoch: 240, Loss: 0.4616447389125824
Epoch: 250, Loss: 0.46129584312438965
Epoch: 260, Loss: 0.4609562158584595
Epoch: 270, Loss: 0.4606261253356933

### Step 7 — Observing the results


---



During training, the loss gradually decreases, showing that the model is learning to classify the data correctly.  
After enough epochs, the loss should stabilize around a small value (close to 0), indicating good performance.  

This confirms that the network can differentiate between male and female based on height and weight.


In [15]:

# Test on a single example, just to check
if model(torch.tensor([160.0, 50.0])).argmax().item() == 0:
  print('Female')
else:
  print('Male')

Female
